In [4]:
import sqlite3
import pandas as pd

In [5]:
_DB_PATH = "./db.sqlite"

In [6]:
def db_con() -> sqlite3.Connection:
    con: sqlite3.Connection = sqlite3.connect(_DB_PATH)
    con.row_factory = sqlite3.Row
    return con


def query(sql: str) -> pd.DataFrame:
    return pd.read_sql(sql, db_con())

In [8]:
query("select * from keywords limit 3")

,id,chunk_id,word,count
0,1,1,*,12
1,2,1,福岡,1
2,3,1,中央,1


In [22]:
query("select ln(1.0)")

DatabaseError: Execution failed on sql 'select ln(1.0)': no such function: ln

log関数が使えなくて積んでいる。tf, dfだけ求めてpythonでやるのが良いかも

In [18]:
query("""
with document_length as (
    select
      chunk_id,
      sum(count) as length
    from
      keywords
    group by
      chunk_id
), term_frequency as (
    select
      k.chunk_id,
      k.word,
      k.count,
      dl.length,
      (1.0 * k.count) / dl.length as term_frequency
    from
      keywords k
        inner join
      document_length dl
        on k.chunk_id = dl.chunk_id
), document_count as (
    select
      word,
      count(distinct chunk_id) as document_count
    from
      keywords
    group by
      word      
), inverse_document_frequency as (
    select
      word,
      document_count,
      -log(
        1.0 * document_count / (
          select count(1) from chunks
        )
      ) as inverse_document_frequency
    from
      document_count
)
select
  *
from
  inverse_document_frequency
""")

DatabaseError: Execution failed on sql '
with document_length as (
    select
      chunk_id,
      sum(count) as length
    from
      keywords
    group by
      chunk_id
), term_frequency as (
    select
      k.chunk_id,
      k.word,
      k.count,
      dl.length,
      (1.0 * k.count) / dl.length as term_frequency
    from
      keywords k
        inner join
      document_length dl
        on k.chunk_id = dl.chunk_id
), document_count as (
    select
      word,
      count(distinct chunk_id) as document_count
    from
      keywords
    group by
      word      
), inverse_document_frequency as (
    select
      word,
      document_count,
      -log(
        1.0 * document_count / (
          select count(1) from chunks
        )
      ) as inverse_document_frequency
    from
      document_count
)
select
  *
from
  inverse_document_frequency
': no such function: log